# Visualise NIHSS after preprocessing but in real scale

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
preprocessed_features_path = '/Users/jk1/temp/mimic/preprocessing/test/mimic_prepro_29122022_173530/preprocessed_features_29122022_173530.csv'
admission_notes_data_path = '/Users/jk1/OneDrive - unige.ch/stroke_research/geneva_stroke_unit_dataset/data/mimic_data/combined_notes_labels.xlsx'
normalisation_params_path = '/Users/jk1/temp/mimic/preprocessing/test/mimic_prepro_29122022_163718/logs_29122022_163718/reference_population_normalisation_parameters.csv'

In [ ]:
preprocessed_features_df = pd.read_csv(preprocessed_features_path)
normalisation_params_df = pd.read_csv(normalisation_params_path)

In [ ]:
admission_notes_data_df = pd.read_excel(admission_notes_data_path)

In [ ]:
def reverse_normalisation_for_subj(norm_subj_df, normalisation_parameters_df):
    subj_df = norm_subj_df.copy()
    for variable in normalisation_parameters_df.variable.unique():
        if variable not in subj_df.sample_label.unique():
            continue

        temp = subj_df[subj_df.sample_label == variable].value.copy()
        std = normalisation_parameters_df[normalisation_parameters_df.variable == variable].original_std.iloc[0]
        mean = normalisation_parameters_df[normalisation_parameters_df.variable == variable].original_mean.iloc[0]
        temp = (temp * std) + mean
        subj_df.loc[subj_df.sample_label == variable, 'value'] = temp

    return subj_df

In [ ]:
preprocessed_features_df.head()

In [ ]:
pa_id = np.random.choice(preprocessed_features_df['case_admission_id'].unique())
pa_id

In [ ]:
admission_notes_data_df['case_admission_id'] = admission_notes_data_df.hadm_id.astype(str) + '_' + admission_notes_data_df.icustay_id.astype(str)
admission_notes_data_df.head()

In [ ]:
pa_id = '120793_201738'

In [ ]:
vital_name = 'median_NIHSS'
subj_denorm_data = reverse_normalisation_for_subj(preprocessed_features_df[preprocessed_features_df.case_admission_id == pa_id], normalisation_params_df)
subj_admission_nihss = admission_notes_data_df[admission_notes_data_df.case_admission_id == pa_id]['admission NIHSS'].values[0]
temp = subj_denorm_data[(subj_denorm_data.sample_label.isin([vital_name]))].copy()
ax = sns.scatterplot(x='relative_sample_date_hourly_cat', y='value', data=temp, hue='value', legend=False)
ax.set_xlabel('Hours from admission')
ax.set_ylabel(vital_name)
ax.tick_params(axis="x", rotation=45)
ax.set_ylim(0, 41)
ax.set_title(f'{vital_name}, with admission NIHSS at {subj_admission_nihss}')

plt.show()

High admission NIHSS values do not appear as they are filtered by winsorizing (using the params obtained from reference population)